<h1>Run SEIRHUD</h1>

In [1]:
from model import SEIRHUD
import csv
import numpy as np
import pandas as pd
import time
import warnings
from tqdm import tqdm
import multiprocessing as mp
warnings.filterwarnings('ignore')

In [2]:
data = pd.read_csv("../data/data.csv")
data.head()

,dates,ts0,infec,leitos,uti,dth,dthcm
0,06/03/2020,8,1,0,0,0,0
1,07/03/2020,9,2,0,0,0,0
2,08/03/2020,10,2,0,0,0,0
3,09/03/2020,11,2,0,0,0,0
4,10/03/2020,12,2,0,0,0,0


In [3]:
#Define empty lists to recive results
ypred = []
dpred = []
upred = []
hpred = []
spred = []
epred = []
beta1 = []
beta2 = []
gammaH = []
gammaU = []
delta = []
ia0 = []
t1 = []
e0 = []
is0 = []

In [4]:
#define fixed parameters:
kappa = 1/4
p = 0.2
gammaA = 1/3.5
gammaS = 1/4
muH = 0.15
muU = 0.4
xi = 0.53 
omega_U = 0.29 
omega_H = 0.14
N = 14873064
bound = ([0,0,0,1/14,1/14,0,0.05,0,0,0],
         [1.5,1,30,1/5,1/5,1,0.35,10/N,10/N,10/N])

In [5]:
model = SEIRHUD(N,nCores=1)

In [6]:
model.fit(x=range(1,len(data.infec)+1),y=data.infec,d=data.dth,fittingByCumulativeCases=False,kappa=kappa,p = p,gammaA = gammaA, gammaS = gammaS, muH = muH,muU = muU,xi = xi,omegaU = omega_U,omegaH=omega_H ,bound= bound)

pyswarms.single.local_best: 100%|██████████|1000/1000


posi=[8.07619839e-01 3.87377564e-01 1.90963469e+00 1.26973548e-01
 1.49223265e-01 6.22308661e-02 5.95648369e-02 9.40070696e-08
 6.29265512e-07 2.89067908e-07]
16.183463138273172


In [7]:
model.save("../results/model1.dat")
model.predict(200)
model.save("../results/model2.dat")

In [ ]:
model = SEIRHUD.load("../results/model2.dat")
model.computeCI(times=500, level=0.95)


0



pyswarms.single.global_best:  57%|█████▊    |575/1000, best_cost=1.26e+3

In [ ]:
model.save("../results/model3.dat")

In [ ]:
#Exprort parametes
parameters = pd.DataFrame.from_dict({"beta1": model.lBeta[0],
                                     "beta2": model.lBeta[1],
                                     "gammaH": model.lGammaH,
                                     "gammaU": model.lGammaU,
                                     "delta": model.LDelta,
                                     "ia0":model.lIa0,
                                     "e0": model.lE0,
                                     "t1": model.lDayBetaChange[0],
                                     "is0":model.lIs0})
parameters.to_csv("../results/Parameters.csv", index = False)